In [ ]:
# Paths to the uploaded zip file and extraction location
zip_file = '/content/drive/MyDrive/archive.zip' # Update with the correct path to your zip file
output_dir = '/content/extracted_files'  # Specify the directory to extract to


In [ ]:
import os
# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

In [ ]:
import os
import zipfile # Import the zipfile module


# Unzip the file
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(output_dir)

print(f"Files extracted to {output_dir}")

Files extracted to /content/extracted_files


In [ ]:
# Load Dataset
image_folder = '/content/extracted_files/Segmented Medicinal Leaf Images'  # Change to your dataset path
labels = []
features = []


In [ ]:
for label in os.listdir(image_folder):
    class_folder = os.path.join(image_folder, label)
    if os.path.isdir(class_folder):
        for image_file in os.listdir(class_folder):
            image_path = os.path.join(class_folder, image_file)
            features.append(extract_features(image_path, base_model))
            labels.append(label)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━

In [ ]:
import numpy as np
import os
import cv2
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

In [ ]:
def load_images_from_folder(folder, img_size=(224, 224)):
    images = []
    labels = []
    for label in os.listdir(folder):
        label_path = os.path.join(folder, label)
        if os.path.isdir(label_path):
            for file in os.listdir(label_path):
                img_path = os.path.join(label_path, file)
                img = cv2.imread(img_path)
                if img is not None:
                    img = cv2.resize(img, img_size)
                    images.append(img)
                    labels.append(label)
    return np.array(images), np.array(labels)

In [ ]:
# Define dataset path
dataset_path = "/content/extracted_files/Segmented Medicinal Leaf Images"  # Adjust the path accordingly
X, y = load_images_from_folder(dataset_path)


In [ ]:
# Preprocess images for VGG16
X = preprocess_input(X)

# Encode labels
le = LabelEncoder()
y = le.fit_transform(y)

In [ ]:
# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Load VGG16 model (pretrained)
vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
# Extract features
X_train_features = vgg16.predict(X_train)
X_test_features = vgg16.predict(X_test)

46/46 ━━━━━━━━━━━━━━━━━━━━ 1001s 22s/step
12/12 ━━━━━━━━━━━━━━━━━━━━ 240s 20s/step


In [ ]:
# Flatten features
X_train_features = X_train_features.reshape(X_train_features.shape[0], -1)
X_test_features = X_test_features.reshape(X_test_features.shape[0], -1)

In [ ]:
# Train SVM model
svm = SVC(kernel='linear', probability=True)
svm.fit(X_train_features, y_train)

SVC(kernel='linear', probability=True)

In [ ]:
joblib.dump(svm, 'svm_model.pkl')
joblib.dump(le, 'label_encoder.pkl')

['label_encoder.pkl']

In [ ]:
# Make predictions
y_pred = svm.predict(X_test_features)

In [ ]:
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"SVM Accuracy: {accuracy * 100:.2f}%")

SVM Accuracy: 97.00%


In [ ]:
from sklearn.metrics import classification_report

# Generate Classification Report
report = classification_report(y_test, y_pred, target_names=le.classes_)
print("Classification Report:\n")
print(report)


Classification Report:

                         precision    recall  f1-score   support

            Arive-Dantu       0.90      0.95      0.92        19
                 Basale       0.96      1.00      0.98        22
                  Betel       1.00      1.00      1.00        14
          Crape_Jasmine       1.00      1.00      1.00        11
                  Curry       0.88      0.88      0.88         8
              Drumstick       1.00      0.94      0.97        17
              Fenugreek       0.91      1.00      0.95        10
                  Guava       1.00      1.00      1.00        14
               Hibiscus       1.00      0.77      0.87        13
           Indian_Beech       1.00      1.00      1.00         9
         Indian_Mustard       1.00      1.00      1.00        11
              Jackfruit       1.00      1.00      1.00        15
Jamaica_Cherry-Gasagase       1.00      1.00      1.00         7
                  Jamun       1.00      0.89      0.94         9


In [ ]:
import cv2
import numpy as np
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
import joblib

# Load pre-trained models
vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
svm = joblib.load('svm_model.pkl')  # Assuming you've saved the SVM model
le = joblib.load('label_encoder.pkl')  # Assuming you've saved the LabelEncoder

def predict_leaf_image(image_path):
    # Load and preprocess the image
    img = cv2.imread(image_path)
    img = cv2.resize(img, (224, 224))
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)

    # Feature extraction
    features = vgg16.predict(img)
    features = features.reshape(1, -1)

    # Predict using SVM
    prediction = svm.predict(features)
    probability = svm.predict_proba(features)

    # Classify as Medicinal or Non-Medicinal
    confidence = np.max(probability) * 100
    if confidence > 50:  # You can adjust this threshold
        result = "Medicinal"
    else:
        result = "Non-Medicinal"

    print(f"Prediction: {result} ({confidence:.2f}% confidence)")

# Example usage
image_path = '/content/extracted_files/Segmented Medicinal Leaf Images/Arive-Dantu/AV-S-001.jpg'  # Replace with your image path
predict_leaf_image(image_path)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 728ms/step
Prediction: Medicinal (50.62% confidence)


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Assuming y_test contains the true labels for the test set
y_true = y_test

accuracy = accuracy_score(y_true, y_pred)
# Use 'micro', 'macro', or 'weighted' for multi-class
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')

# Print the results
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

Accuracy: 0.9700272479564033
Precision: 0.9730555081165198
Recall: 0.9700272479564033
F1 Score: 0.970321492531713
